In [1]:
import numpy as np
from scipy.stats import t, norm
from math import atanh, pow
from numpy import tanh

from scipy.stats import binom_test

In [2]:
def dependent_corr(xy, xz, yz, n, twotailed=True):
    d = xy - xz
    determin = max(1 - xy * xy - xz * xz - yz * yz + 2 * xy * xz * yz, 0)
    av = (xy + xz)/2
    cube = (1 - yz) * (1 - yz) * (1 - yz)

    t2 = d * np.sqrt((n - 1) * (1 + yz)/(((2 * (n - 1)/(n - 3)) * determin + av * av * cube)))
    p = 1 - t.cdf(abs(t2), n - 3)

    if twotailed:
        p *= 2

    return t2, p

$$\textit{stest-p}(\hat{r}_{AT}, \hat{r}_{BT}, r_{AB}, p_0, n)
{\implies} \textit{pval} < p_0$$

$$\sigma_{p_0}^r =
  \min\{\sigma | \forall\, 0 {<} r' {<} 1\, \textit{stest-p}(r',
  \min(r'+\sigma, 1), r, p_0, n) \} $$

In [3]:
def stest_p(r_at, r_bt, r_ab, p0, n):
    t, p_val = dependent_corr(r_at, r_bt, r_ab, n, twotailed=False)
    
    return p_val < p0

In [4]:
def stest_p_test(σ, r=0.9, p0=0.05, n=3000):
    return all(stest_p(r_, min(r_ + σ, 1), r, p0, n) for r_ in np.linspace(0, 1 - σ, num=100))

In [5]:
def find_sigma(r=0.9, p0=0.05, n=3000):
    for σ in np.arange(0, 1, 0.000001):
        if stest_p_test(σ, r, p0, n):
            return σ

In [6]:
assert (
    stest_p_test(0.013), stest_p_test(0.013435), stest_p_test(0.014), stest_p_test(0.182), stest_p_test(0.183)
) == (
    False, True, True, True, True
)

In [7]:
def Search(left=0, right=1, eps=0.00001):
    while right - left > eps:
        half = (left + right) / 2
        is_smaller = yield half
        
        if is_smaller:
            right = half
        else:
            left = half

In [8]:
assert not stest_p_test(0.012, 0.9, 0.05, 3000)

In [9]:
assert stest_p_test(0.01345, 0.9, 0.05, 3000)
assert stest_p_test(1, 0.9, 0.05, 3000)

In [10]:
def binary_find_sigma(r=0.9, p0=0.05, n=3000):
    search = Search()
    sigma = next(search)
    while True:
        try:
            sigma = search.send(stest_p_test(sigma, r, p0, n))
        except StopIteration:
            return sigma

In [11]:
%time assert '{:.1%}'.format(find_sigma(0.9, 0.05, 3000)) == '1.3%'

CPU times: user 1.66 s, sys: 9.64 ms, total: 1.67 s
Wall time: 1.67 s


In [12]:
%time assert '{:.1%}'.format(binary_find_sigma()) == '1.3%'

CPU times: user 158 ms, sys: 4.21 ms, total: 162 ms
Wall time: 165 ms


![alt text](sigmas.png "Title")

In [13]:
rs = 0.5, 0.7, 0.9
p0s = 0.01, 0.05 

In [14]:
datasets = (
    ('MEN', 3000),
    ('RW', 2034),
    ('SCWS', 2003),
    ('SIMLEX', 999),
    ('WS', 353),
    ('MTURK', 287),
    ('WS-REL', 252),
    ('WS_SEM', 203),
    ('RG', 65),
    ('MC', 30),
    ('KS14', 108),
    ('GS11', 200),
)

In [15]:
for dataset, size in datasets:
    print(dataset, ' '.join('{:.3}'.format(binary_find_sigma(r, p0, size)) for p0 in p0s for r in rs))

MEN 0.0425 0.0329 0.019 0.03 0.0233 0.0134
RW 0.0516 0.04 0.0231 0.0365 0.0283 0.0163
SCWS 0.052 0.0403 0.0232 0.0368 0.0285 0.0164
SIMLEX 0.0736 0.057 0.0329 0.0521 0.0404 0.0233
WS 0.124 0.0959 0.0554 0.0877 0.068 0.0393
MTURK 0.137 0.106 0.0615 0.0973 0.0754 0.0436
WS-REL 0.146 0.113 0.0656 0.104 0.0805 0.0465
WS_SEM 0.163 0.126 0.0731 0.116 0.0898 0.0519
RG 0.286 0.223 0.13 0.206 0.16 0.0927
MC 0.418 0.328 0.191 0.307 0.239 0.139
KS14 0.223 0.173 0.1 0.159 0.124 0.0714
GS11 0.164 0.127 0.0737 0.117 0.0905 0.0523


In [21]:
def stest_binom(s, f, p0=0.05):
    return binom_test([s, f], alternative='greater') < p0

In [22]:
def stest_binom_test(sigma, N=28, p0=0.05):
    s = int(sigma * N)
    return all(stest_binom(s + f, f) for f in range(N - s))

In [23]:
def binary_find_btest(p0=0.05, n=28):
    search = Search()
    sigma = next(search)
    while True:
        try:
            sigma = search.send(stest_binom_test(sigma, n, p0))
        except StopIteration:
            return sigma

In [24]:
'{:.2f}'.format(binary_find_btest())

'0.43'

In [25]:
for n in [28, 50, 100, 1000, 2000, 3000]:
    print('{}: {:.3f}'.format(n, binary_find_btest(n=n)))

28: 0.429
50: 0.320
100: 0.230
1000: 0.074
2000: 0.052
3000: 0.043
